In [1]:
from google.colab import drive
import pandas as pd
from transformers import pipeline

In [40]:
#pip install --upgrade torch torchvision transformers

#### Applying roberta-base-sentiment for sentiment classification for the all the comments

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
all_comments = pd.read_csv("/content/drive/My Drive/AI-engineer/comments.csv")
all_comments.head()

,Unnamed: 0,post_id,Created,comment_id,body,created_utc,author_id,score,link_id,parent_id,Keyword
0,0,1jo735c,2025-03-31 16:31:21,mkpjkfy,something fantastic about a fake doctor spammi...,1.743439e+09,Gom_KBull,6755,t3_1jo735c,t3_1jo735c,Medical AI
1,1,1jo735c,2025-03-31 16:56:58,mkpomzc,https://preview.redd.it/4rh19ihv22se1.jpeg?wid...,1.743440e+09,7Hakuna_Matata7,3068,t3_1jo735c,t3_1jo735c,Medical AI
2,2,1jo735c,2025-03-31 16:49:19,mkpn47r,Board certified boob doctor.,1.743440e+09,mysteriousmeatman,607,t3_1jo735c,t3_1jo735c,Medical AI
3,3,1jo735c,2025-03-31 16:22:51,mkphvyc,This should be illegal,1.743438e+09,triple7freak1,7065,t3_1jo735c,t3_1jo735c,Medical AI
4,4,1jo735c,2025-03-31 16:35:45,mkpkfde,We’re really about to enter a grifter’s renais...,1.743439e+09,Murmido,218,t3_1jo735c,t3_1jo735c,Medical AI


In [4]:
all_comments.shape

(20792, 11)

In [6]:
# Load the sentiment analysis pipeline
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    device = 0,
    truncation=True,
    max_length=512
)

batch_size = 32
sentiments = []

for i in range(0, len(all_comments), batch_size):
    batch = all_comments['body'][i:i + batch_size].tolist()
    sentiments.extend(sentiment_pipeline(batch))

# Add results to DataFrame
all_comments['sentiment_label'] = [result['label'] for result in sentiments]
all_comments['sentiment_score'] = [result['score'] for result in sentiments]

# Mapping sentiment labels to numeric values (0, 1, 2)
sentiment_mapping = {
    'LABEL_0': 0,  # Negative
    'LABEL_1': 1,  # Neutral
    'LABEL_2': 2   # Positive
}

# Map the sentiment labels to your specified numeric values
all_comments['sentiment'] = all_comments['sentiment_label'].map(sentiment_mapping)

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
all_comments.head()

,Unnamed: 0,post_id,Created,comment_id,body,created_utc,author_id,score,link_id,parent_id,Keyword,sentiment_label,sentiment_score,sentiment
0,0,1jo735c,2025-03-31 16:31:21,mkpjkfy,something fantastic about a fake doctor spammi...,1.743439e+09,Gom_KBull,6755,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_0,0.505554,0
1,1,1jo735c,2025-03-31 16:56:58,mkpomzc,https://preview.redd.it/4rh19ihv22se1.jpeg?wid...,1.743440e+09,7Hakuna_Matata7,3068,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_1,0.768416,1
2,2,1jo735c,2025-03-31 16:49:19,mkpn47r,Board certified boob doctor.,1.743440e+09,mysteriousmeatman,607,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_1,0.645257,1
3,3,1jo735c,2025-03-31 16:22:51,mkphvyc,This should be illegal,1.743438e+09,triple7freak1,7065,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_0,0.900811,0
4,4,1jo735c,2025-03-31 16:35:45,mkpkfde,We’re really about to enter a grifter’s renais...,1.743439e+09,Murmido,218,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_1,0.629456,1


In [41]:
# Randomly sample 3500 non-neutral comments for hand labelling

new_data = all_comments[all_comments["sentiment"] != 1] #non-neutral
sample_df = new_data.sample(n=3500, random_state=42)
sample_df.to_csv("sample1.csv")

## Training an ML model on the hand labelelled dataset

In [9]:
import pandas as pd
import string
import nltk
import joblib
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import LatentDirichletAllocation
from xgboost import XGBClassifier

#### Data processing steps

In [10]:
df = pd.read_csv("/content/drive/My Drive/AI-engineer/sample1.csv")
df = df.drop(df.columns[[0]], axis=1)
df.head()

,post_id,Created,comment_id,body,created_utc,author_id,score,link_id,parent_id,Keyword,sentiment_label,sentiment_score,sentiment,length,mysentiment
0,1k1dxpp,17/4/2025 22:34,mnnzrp6,It's helping me to be a better partner and a b...,1.744929e+09,capodecina2,2.0,t3_1k1dxpp,t3_1k1dxpp,Medical AI,LABEL_2,0.985333,2.0,338.0,2
1,1j16q7u,1/3/2025 20:50,mfhqyr6,Excellent and level headed view. As a low key ...,1.740862e+09,Any_Needleworker_273,15.0,t3_1j16q7u,t3_1j16q7u,Medical AI,LABEL_2,0.602790,2.0,688.0,2
2,1iuu7ec,21/2/2025 22:45,me2mqsb,"Fuck it, put it in charge.",1.740178e+09,drizel,1.0,t3_1iuu7ec,t3_1iuu7ec,Medical AI,LABEL_0,0.898113,0.0,26.0,0
3,w5xrxe,23/7/2022 14:11,ihboknk,Who is this AL fella and why is he the answer ...,1.658585e+09,DimSumDad72,-2.0,t3_w5xrxe,t3_w5xrxe,Medical AI,LABEL_0,0.779723,0.0,66.0,0
4,1cycbkq,22/5/2024 22:34,l58nrf3,No thank you! I prefer this shell.,1.716417e+09,NaN,2.0,t3_1cycbkq,t3_1cycbkq,Medical AI,LABEL_2,0.675599,2.0,34.0,0


In [11]:
labelled_df = df.iloc[:3500]
labelled_df.loc[:, 'mysentiment'] = labelled_df['mysentiment'].fillna(labelled_df['sentiment'])
labelled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   post_id          3500 non-null   object 
 1   Created          3500 non-null   object 
 2   comment_id       3500 non-null   object 
 3   body             3500 non-null   object 
 4   created_utc      3500 non-null   float64
 5   author_id        3332 non-null   object 
 6   score            3500 non-null   float64
 7   link_id          3500 non-null   object 
 8   parent_id        3500 non-null   object 
 9   Keyword          3500 non-null   object 
 10  sentiment_label  3500 non-null   object 
 11  sentiment_score  3500 non-null   float64
 12  sentiment        3500 non-null   float64
 13  length           3500 non-null   float64
 14  mysentiment      3500 non-null   object 
dtypes: float64(5), object(10)
memory usage: 410.3+ KB


In [21]:
#Make the data consistent
labelled_df['mysentiment'] = labelled_df['mysentiment'].replace(
    {0.0: 0, 2.0: 1, 2 : 1, 'o': 0, '-': 0, ' ': 0}
)

labelled_df.loc[:, 'mysentiment'] = labelled_df['mysentiment'].astype(int)

print(labelled_df['mysentiment'].value_counts())

mysentiment
0    2258
1    1242
Name: count, dtype: int64


<ipython-input-21-6730ed4bc863>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labelled_df['mysentiment'] = labelled_df['mysentiment'].replace(
<ipython-input-21-6730ed4bc863>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labelled_df['mysentiment'] = labelled_df['mysentiment'].replace(


In [22]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [23]:
# Text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)

    # Removing stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Rejoining tokens
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Apply preprocessing
labelled_df.loc[:, 'cleaned_text'] = labelled_df['body'].apply(preprocess_text)

In [24]:
labelled_df.shape

(3500, 16)

### Model training

In [25]:
#Data splitting

X = labelled_df['cleaned_text']
y = labelled_df['mysentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Multinomial Naive Bayes": MultinomialNB(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

vectorizers = {
    "Count Vectorizer": CountVectorizer(max_features=5000),
    "TF-IDF Vectorizer": TfidfVectorizer(max_features=5000)
}

# Function to train and evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, vectorizer, use_lda = False):
    # Apply the vectorizer
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    # If using LDA, fit the model separately
    if use_lda == False:
        # Fit the model
        model.fit(X_train_vec, y_train)
        y_pred = model.predict(X_test_vec)

        # Print results
        print(f"{model.__class__.__name__} with {vectorizer.__class__.__name__} Classification Report:")
        print(classification_report(y_test, y_pred))
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("\n")

    # If using LDA, transform the data
    else:

        lda = LatentDirichletAllocation(n_components=5, random_state=42)
        X_train_lda = lda.fit_transform(X_train_vec)
        X_test_lda = lda.transform(X_test_vec)

        model.fit(X_train_lda, y_train)
        y_pred = model.predict(X_test_lda)

        print(f"{model.__class__.__name__} with LDA Classification Report:")
        print(classification_report(y_test, y_pred))
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("\n")

In [27]:
# Evaluate each model with Count Vectorizer and TF-IDF
for model_name, model in models.items():
    for vectorizer_name, vectorizer in vectorizers.items():
        evaluate_model(model, X_train, X_test, y_train, y_test, vectorizer)

LogisticRegression with CountVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       441
           1       0.75      0.66      0.70       259

    accuracy                           0.79       700
   macro avg       0.78      0.76      0.77       700
weighted avg       0.79      0.79      0.79       700

Accuracy: 0.7914285714285715


LogisticRegression with TfidfVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.99      0.87       441
           1       0.96      0.51      0.67       259

    accuracy                           0.81       700
   macro avg       0.87      0.75      0.77       700
weighted avg       0.84      0.81      0.79       700

Accuracy: 0.8114285714285714


SVC with CountVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.97      0.85       441
 

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:12:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier with CountVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.93      0.84       441
           1       0.81      0.54      0.65       259

    accuracy                           0.78       700
   macro avg       0.79      0.73      0.75       700
weighted avg       0.79      0.78      0.77       700

Accuracy: 0.7842857142857143




/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:12:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier with TfidfVectorizer Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.93      0.85       441
           1       0.82      0.56      0.67       259

    accuracy                           0.79       700
   macro avg       0.80      0.75      0.76       700
weighted avg       0.80      0.79      0.78       700

Accuracy: 0.7942857142857143




In [28]:
# Evaluate each model with LDA
for model_name, model in models.items():
    count_vectorizer = CountVectorizer(max_features=5000)
    evaluate_model(model, X_train, X_test, y_train, y_test, count_vectorizer, use_lda=True)

LogisticRegression with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.63      1.00      0.77       441
           1       0.00      0.00      0.00       259

    accuracy                           0.63       700
   macro avg       0.32      0.50      0.39       700
weighted avg       0.40      0.63      0.49       700

Accuracy: 0.63




/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVC with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.95      0.77       441
           1       0.59      0.13      0.21       259

    accuracy                           0.64       700
   macro avg       0.62      0.54      0.49       700
weighted avg       0.63      0.64      0.56       700

Accuracy: 0.6442857142857142


RandomForestClassifier with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.82      0.74       441
           1       0.52      0.34      0.41       259

    accuracy                           0.64       700
   macro avg       0.60      0.58      0.58       700
weighted avg       0.62      0.64      0.62       700

Accuracy: 0.6414285714285715


MultinomialNB with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.63      1.00      0.77       441
           1       0.00      0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGBClassifier with LDA Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.81      0.74       441
           1       0.53      0.36      0.42       259

    accuracy                           0.64       700
   macro avg       0.61      0.58      0.58       700
weighted avg       0.63      0.64      0.63       700

Accuracy: 0.6442857142857142




/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:13:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


#### Performing hyperparameter tuning for the 2 best models (LR + TFIDF, MNB + TF) using Grid Search CV

In [29]:
vectorizer_tf = CountVectorizer(max_features=5000)
X_train_tf = vectorizer_tf.fit_transform(X_train)
X_test_tf = vectorizer_tf.transform(X_test)

vectorizer_tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

In [30]:
logistic_param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}

# Grid Search for Logistic Regression with TF-IDF
logistic_grid_search = GridSearchCV(LogisticRegression(max_iter=1000),
                                     logistic_param_grid,
                                     scoring='accuracy',
                                     cv=5)
logistic_grid_search.fit(X_train_tfidf, y_train)

# Best Logistic Regression Model
best_logistic_model = logistic_grid_search.best_estimator_
y_pred_logistic = best_logistic_model.predict(X_test_tf)

# Evaluation
print("Best Logistic Regression Parameters:", logistic_grid_search.best_params_)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_logistic))
print(accuracy_score(y_test, y_pred_logistic))

Best Logistic Regression Parameters: {'C': 10, 'solver': 'lbfgs'}
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       441
           1       0.68      0.76      0.72       259

    accuracy                           0.78       700
   macro avg       0.76      0.77      0.77       700
weighted avg       0.78      0.78      0.78       700

0.7771428571428571


In [31]:
# Grid Search for Multinomial Naive Bayes with TF

nb_param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'fit_prior': [True, False]
}

nb_grid_search = GridSearchCV(MultinomialNB(),
                               nb_param_grid,
                               scoring='accuracy',
                               cv=5)
nb_grid_search.fit(X_train_tf, y_train)

# Best Naive Bayes Model
best_nb_model = nb_grid_search.best_estimator_
y_pred_nb = best_nb_model.predict(X_test_tfidf)

print("Best Multinomial Naive Bayes Parameters:", nb_grid_search.best_params_)
print("Multinomial Naive Bayes Classification Report:")
print(classification_report(y_test, y_pred_nb))
print(accuracy_score(y_test, y_pred_nb))

Best Multinomial Naive Bayes Parameters: {'alpha': 0.1, 'fit_prior': True}
Multinomial Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.94      0.85       441
           1       0.85      0.56      0.67       259

    accuracy                           0.80       700
   macro avg       0.82      0.75      0.76       700
weighted avg       0.81      0.80      0.79       700

0.7985714285714286


In [32]:
# Define hyperparameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid Search for Random Forest
rf_grid_search = GridSearchCV(RandomForestClassifier(),
                               rf_param_grid,
                               scoring='accuracy',
                               cv=5)
rf_grid_search.fit(X_train_tfidf, y_train)

# Best Random Forest Model
best_rf_model = rf_grid_search.best_estimator_
y_pred_rf = best_rf_model.predict(X_test_tfidf)

# Evaluation
print("Best Random Forest Parameters:", rf_grid_search.best_params_)
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

Best Random Forest Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.93      0.85       441
           1       0.82      0.54      0.65       259

    accuracy                           0.79       700
   macro avg       0.80      0.73      0.75       700
weighted avg       0.79      0.79      0.77       700



#### Applying the best model to all the comments

In [36]:
# Train Multinomial Naive Bayes model with default parameters
nb_model = MultinomialNB()
nb_model.fit(X_train_tf, y_train)

nb_pred = nb_model.predict(X_test_tf)
print("Training Accuracy:", accuracy_score(y_test, nb_pred))

Training Accuracy: 0.8242857142857143


In [37]:
# Transform the new data using the tf vectorizer
all_comments['cleaned_text'] = all_comments['body'].apply(preprocess_text)
X_all_comments_tf = vectorizer_tf.transform(all_comments['cleaned_text'])

# Predict using the trained model
predictions = nb_model.predict(X_all_comments_tf)

# Add mysentiment to all_comments
all_comments['mysentiment'] = predictions

In [38]:
all_comments.head()

,Unnamed: 0,post_id,Created,comment_id,body,created_utc,author_id,score,link_id,parent_id,Keyword,sentiment_label,sentiment_score,sentiment,cleaned_text,mysentiment
0,0,1jo735c,2025-03-31 16:31:21,mkpjkfy,something fantastic about a fake doctor spammi...,1.743439e+09,Gom_KBull,6755,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_0,0.505554,0,something fantastic fake doctor spamming b00b,0
1,1,1jo735c,2025-03-31 16:56:58,mkpomzc,https://preview.redd.it/4rh19ihv22se1.jpeg?wid...,1.743440e+09,7Hakuna_Matata7,3068,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_1,0.768416,1,httpspreviewreddit4rh19ihv22se1jpegwidth1026fo...,0
2,2,1jo735c,2025-03-31 16:49:19,mkpn47r,Board certified boob doctor.,1.743440e+09,mysteriousmeatman,607,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_1,0.645257,1,board certified boob doctor,0
3,3,1jo735c,2025-03-31 16:22:51,mkphvyc,This should be illegal,1.743438e+09,triple7freak1,7065,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_0,0.900811,0,illegal,0
4,4,1jo735c,2025-03-31 16:35:45,mkpkfde,We’re really about to enter a grifter’s renais...,1.743439e+09,Murmido,218,t3_1jo735c,t3_1jo735c,Medical AI,LABEL_1,0.629456,1,’ really enter grifter ’ renaissance ’,0


In [39]:
# Save the final labelled version
all_comments.to_csv("comments_labelled.csv")